# Topics
Compute the topics

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts
from phoenix.common import utils
from phoenix.tag import topic
from phoenix.tag.topic import single_feature_match as sfm
from phoenix.tag.topic import single_feature_match_topic_config as sfm_topic_config

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parametrise the run execution date.
# Format of the run date
RUN_DATE_FORMAT = "%Y-%m-%d"
# This can be overwritten at execution time by Papermill to enable historic runs and backfills etc.
# RUN_DATE = datetime.datetime.today().strftime(RUN_DATE_FORMAT)
RUN_DATE = "2021-06-10"
# Set Artefacts URL
ARTIFACTS_BASE_URL = f"{artifacts.urls.get_local()}{RUN_DATE}/"

# Input
ALL_FEATURES = artifacts.dataframes.url(ARTIFACTS_BASE_URL, "all_features")

In [ ]:
# Display params.
print(
ARTIFACTS_BASE_URL,
ALL_FEATURES,
RUN_DATE,
sep='\n',
)

In [ ]:
# %env DASK_CLUSTER_IP=tcp://127.0.0.1:41037

In [ ]:
utils.dask_global_init()

In [ ]:
all_features_df = artifacts.dataframes.get(ALL_FEATURES).dataframe

In [ ]:
all_features_df.head()

In [ ]:
topic_config_df = sfm.get_topic_config()

In [ ]:
topic_config_df.head()

In [ ]:
topics_df = sfm_topic_config.get_topics(topic_config_df, all_features_df)

In [ ]:
topics_df.head()

In [ ]:
topics_df.shape[0]

In [ ]:
objects_topics = topic.get_object_topics(topics_df)

In [ ]:
objects_topics.head()

In [ ]:
objects_topics.shape[0]

In [ ]:
_ = artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "topics"), topics_df)

In [ ]:
with tentaclio.open(ARTIFACTS_BASE_URL + f"topics.csv", "w") as fb:
    topics_df.to_csv(fb)

In [ ]:
with tentaclio.open(ARTIFACTS_BASE_URL + f"objects_topics.csv", "w") as fb:
    objects_topics.to_csv(fb)